# Run RPPG

In [5]:
import sys

import matplotlib.pyplot as plt
from PyQt5.QtWidgets import QApplication
from yarppg.ui import MainWindow
from yarppg.rppg import RPPG
from yarppg.rppg.processors import ColorMeanProcessor, FilteredProcessor, PosProcessor
from yarppg.rppg.hr import HRCalculator
from yarppg.rppg.filters import get_butterworth_filter
from yarppg.ui.cli import (get_detector, get_mainparser, get_processor,
                           parse_frequencies)
from yarppg.rppg.roi.roi_detect import FaceMeshDetector

In [6]:
parser = get_mainparser()
app = QApplication(sys.argv)

roi_detector = FaceMeshDetector(draw_landmarks=False)

digital_lowpass = get_butterworth_filter(30, 1.5)
hr_calc = HRCalculator(parent=app, update_interval=30, winsize=300,
                           filt_fun=lambda vs: [digital_lowpass(v) for v in vs])

winsize = 250
processor = PosProcessor(winsize)

cutoff = parse_frequencies('0.5,2')
if cutoff is not None:
    digital_bandpass = get_butterworth_filter(30, cutoff, "bandpass")
    processor = FilteredProcessor(processor, digital_bandpass)

# 'cohface/1/3/data.avi'
rppg = RPPG(roi_detector=roi_detector,
                video=0,
                hr_calculator=hr_calc,
                parent=app,
                )
rppg.add_processor(processor)

for c in "rgb":
    rppg.add_processor(ColorMeanProcessor(channel=c, winsize=1))


rppg.output_filename = 'data.csv'


win = MainWindow(app=app,
                rppg=rppg,
                winsize=(1000, 400),
                legend=True,
                graphwin=300,
                blur_roi=-1,
                )
for i in range(3):
    win.set_pen(index=i+1, color="rgb"[i], width=1)

x = win.execute()

TEST
frame: [[[255 230 168]
  [255 231 169]
  [255 233 170]
  ...
  [ 85  66  43]
  [ 92  71  49]
  [ 88  66  44]]

 [[255 230 168]
  [255 231 169]
  [255 230 168]
  ...
  [ 82  62  39]
  [ 85  64  41]
  [ 85  64  41]]

 [[255 232 170]
  [255 230 168]
  [255 230 168]
  ...
  [ 90  70  47]
  [ 91  72  47]
  [ 91  72  47]]

 ...

 [[149 131  87]
  [155 137  93]
  [157 139  95]
  ...
  [172 143 109]
  [174 146 111]
  [170 141 106]]

 [[149 131  87]
  [163 145 101]
  [169 151 107]
  ...
  [172 143 109]
  [170 141 107]
  [170 141 107]]

 [[153 135  91]
  [157 139  95]
  [156 138  94]
  ...
  [157 128  95]
  [153 125  91]
  [160 132  98]]] len:  720
frame: [[[255 235 171]
  [255 234 170]
  [255 232 167]
  ...
  [ 83  63  40]
  [ 85  64  41]
  [ 88  66  44]]

 [[255 235 171]
  [255 234 170]
  [255 232 167]
  ...
  [ 84  64  41]
  [ 85  64  41]
  [ 88  66  44]]

 [[255 234 170]
  [255 231 167]
  [255 230 166]
  ...
  [ 79  60  37]
  [ 80  59  37]
  [ 85  64  42]]

 ...

 [[149 131  87]
  [155 

### Convert Prediction into Dataframe

In [ ]:
import pandas as pd
df = pd.read_csv("cohface_1_3.csv")
df = df.rename(columns={"ts": "time", "p0": "pulse_pred"})

# Reading Ground Truth of my_groundtruth

In [ ]:
from datetime import datetime
import pandas as pd
from numpy import genfromtxt
from pulse_comparison import *

gt_1_1 = genfromtxt("my_groundtruth/Participant1_1.csv", delimiter=',')
gt_1_2 = genfromtxt("my_groundtruth/Participant1_2.csv", delimiter=',')
gt_2_1 = genfromtxt("my_groundtruth/Participant2_1.csv", delimiter=',')
gt_2_2 = genfromtxt("my_groundtruth/Participant2_2.csv", delimiter=',')
gt_3 = genfromtxt("my_groundtruth/Participant3.csv", delimiter=',')
gt_4 = genfromtxt("my_groundtruth/Participant4.csv", delimiter=',')
gt_5 = genfromtxt("my_groundtruth/Participant5.csv", delimiter=',')

gt_1_1 = gt_1_1[1:]
gt_1_2 = gt_1_2[1:]
gt_2_1 = gt_2_1[1:]
gt_2_2 = gt_2_2[1:]
gt_3 = gt_3[1:]
gt_4 = gt_4[1:]
gt_5 = gt_5[1:]

allDataframes = getDataframePeaks([gt_1_1, gt_1_2, gt_2_1, gt_2_2, gt_3, gt_4, gt_5])
print(allDataframes[1].head())


In [ ]:
from pulse_comparison import *

participant1_start_times   = ["01:00:30:00", "01:01:54:00", "01:03:24:00", "01:04:54:00", "01:06:36:00", "01:08:12:00", "01:09:50:00", "01:11:28:00", "01:12:54:00", "01:14:20:00"]
participant1_2_start_times = ["01:00:21:57", "01:03:52:00", "01:05:27:57"]
participant2_start_times   = ["01:00:30:00", "01:01:58:00", "01:03:24:00", "01:04:50:00", "01:06:26:00", "01:07:56:00", "01:09:22:00", "01:10:48:00", "01:12:18:00", "01:13:44:00"]
participant2_2_start_times = ["01:00:29:00", "01:04:29:00", "01:06:25:00"]
participant3_start_times   = ["00:12:08:00", "00:13:20:00", "00:14:38:00", "00:15:48:00", "00:17:26:00", "00:18:50:00", "00:20:22:00", "00:21:46:00", "00:23:06:00", "00:24:24:00", "00:25:48:00", "00:28:14:00", "00:30:10:00"]
participant4_start_times = ["01:00:34:00", "01:02:08:00", "01:03:26:00", "01:05:02:00", "01:06:32:00", "01:07:52:00", "01:09:34:00", "01:10:52:00", "01:12:16:00", "01:14:04:00", "01:15:52:00", "01:18:28:00", "01:20:56:00"]
participant5_start_times = ["01:00:30:00", "01:02:56:00", "01:04:26:00", "01:05:52:00", "01:07:12:00", "01:08:28:00", "01:09:42:00", "01:11:02:00", "01:12:18:00", "01:13:44:00", "01:15:08:00", "01:16:28:00", "01:17:48:00"]


one_minute_slices_p1 = split_df(participant1_start_times, allDataframes[0]) + split_df(participant1_2_start_times, allDataframes[1])
one_minute_slices_p2 = split_df(participant2_start_times, allDataframes[2]) + split_df(participant2_2_start_times, allDataframes[3])
one_minute_slices_p3 = split_df(participant3_start_times, allDataframes[4])
one_minute_slices_p4 = split_df(participant4_start_times, allDataframes[5])
one_minute_slices_p5 = split_df(participant5_start_times, allDataframes[6])

all_slices = [one_minute_slices_p1, one_minute_slices_p2, one_minute_slices_p3, one_minute_slices_p4, one_minute_slices_p5]

In [ ]:
participant = 1
for one_minute_slices in all_slices:
    one_minute_slices_str = [slice_df.to_csv() for slice_df in one_minute_slices]
    for condition in range(1,14):
        path = "my_groundtruth/{}/{}/data.txt".format(participant, condition)
        text_file = open(path, "w")
        n = text_file.write(one_minute_slices_str[0])
        text_file.close()
    participant+=1

In [ ]:
read_timecode("01:13:44:00",)

# Writing signals into my_signals

# run rPPG on all videos

In [ ]:
participant = 2
for one_minute_slices in all_slices:
    for condition in range(1,14):
            path = "my_signals/{}/{}/data.csv".format(participant, condition)
            text_file = open(path, "w")
            text_file.close()
    participant+=1

In [ ]:
from pulse_comparison import *
run_rPPG(input_file='my_videos/Participant1/Participant1_Condition10.mp4', output_file='my_signals/1/10/data.csv')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


TEST
frame: [[[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [120 125 131]
  [121 124 131]
  [121 124 131]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [120 125 131]
  [121 124 131]
  [121 124 131]]

 [[  1   1   1]
  [  1   1   1]
  [  1   1   1]
  ...
  [123 123 131]
  [123 123 131]
  [123 123 131]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [ 82  91  93]
  [ 80  91  92]
  [ 80  91  92]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [ 78  91  90]
  [ 78  91  92]
  [ 77  90  91]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [ 78  91  90]
  [ 77  90  91]
  [ 77  90  91]]] len:  720
frame: [[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [120 125 131]
  [121 124 131]
  [121 124 131]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [120 125 131]
  [121 124 131]
  [121 124 131]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [123 123 131]
  [123 123 131]
  [123 123 131]]

 ...

 [[  0   0   0]
  [  0 

IndexError: tuple index out of range

### Convert Original Ground Truth to NP Arrays

In [ ]:
import numpy as np
import h5py

hf = h5py.File('cohface/1/3/data.hdf5', 'r')
hf.keys()

pulse_gt = np.array(hf.get('pulse'))
respiration_gt = np.array(hf.get('respiration'))
time_gt = np.array(hf.get('time'))

In [ ]:
df_cleaned_gt = pd.read_csv("CleanerPPG/COHFACE/Cleaned/1_0 PPG.csv")
print(df_cleaned_gt)

# Downsampling Ground Truth to Sampling Rate of RPPG

In [ ]:
from pulse_comparison import *
print('pulse_gt_cleaned_downsampled:')
(pulse_gt_cleaned_downsampled, xnew_cleaned) = downsample(np.asarray(df_cleaned_gt['Signal']),np.asarray(df_cleaned_gt['Time']/1000), 20)
print('\npulse_gt_downsampled:')
(pulse_gt_downsampled, xnew) = downsample(pulse_gt[0:15444], time_gt[0:15444], 20)

In [ ]:
print(len(pulse_gt), '# of samples in pulse_gt')
print((len(pulse_gt_downsampled)), ' # of samples in downsampled pulse_gt')
print(len(df_cleaned_gt['Signal']), '# of samples in cleaned pulse_gt')
print((len(pulse_gt_cleaned_downsampled)), ' # of samples in downsampled cleaned pulse_gt')
print(len(df['pulse_pred']), ' # of samples in predicted pulse')

### Compare cleaned gt and orinal gt, before and after downsampling

In [ ]:
length = 1000
start = 0
q = 13

# pulse_gt_downsampled compared to pulse_gt
plt.plot(time_gt[start:start+length], pulse_gt[start:start+length], '.-', xnew[int(start/q):int((start+length)/q)], pulse_gt_downsampled[int(start/q):int((start+length)/q)], 'o-')
plt.xlabel('Time, Seconds')
plt.legend(['before downsampling', 'downsampled'], loc='best')
plt.title("Original Ground Truth")
plt.show()

# pulse_gt_cleaned_downsampled compared to df_cleaned_gt['Signal']
plt.plot(df_cleaned_gt['Time'][start:start+length]/1000, df_cleaned_gt['Signal'][start:start+length], '.-', xnew_cleaned[int(start/q):int((start+length)/q)], pulse_gt_cleaned_downsampled[int(start/q):int((start+length)/q)], 'o-')
plt.xlabel('Time, Seconds')
plt.legend(['before downsampling', 'downsampled'], loc='best')
plt.title("Cleaned Ground Truth")
plt.show()


In [ ]:
# cut off end of predicted pulse
#df = df[0:1189]

# add ground truth to data frame
#df.insert(2, "pulse_gt", pulse_gt_downsampled, False)


### Peak Detection

In [ ]:
from pulse_comparison import *

df = df[0:1188]

peaks_pred, peaks_pred_indices = get_peaks(df['pulse_pred'])
df.insert(2, 'peaks_pred', peaks_pred, False)
print(len(peaks_pred_indices))

peaks_gt, peaks_gt_indices = get_peaks(pulse_gt_downsampled)
df.insert(2, 'peaks_gt', peaks_gt, False)
print(len([x for x in peaks_gt_indices if x > 250]))

start_frame = 600
window_size = 600

plt.plot(df['pulse_pred'][start_frame:start_frame+window_size])
plt.plot([x for x in peaks_pred_indices if x > start_frame and x < start_frame+window_size], df['pulse_pred'][[x for x in peaks_pred_indices if x > start_frame and x < start_frame+window_size]], "x")
plt.title("Peaks of Prediction BVP")
plt.show()

plt.plot(pulse_gt_downsampled[start_frame:start_frame+window_size])
plt.plot([x -start_frame for x in peaks_gt_indices if x > start_frame and x < start_frame+window_size], pulse_gt_downsampled[[x for x in peaks_gt_indices if x > start_frame and x < start_frame+window_size]], "x")
plt.title("Peaks of Ground Truth BVP")
plt.show()

df[1100:]

### Compare peaks to ground truth

### Calculate Inter-Beat-Intervals (IBI)

In [ ]:
filter_width = 1

IBIs = calculate_IBIs(peaks=df['peaks_pred'], frame_rate=20, filter_width=filter_width)
mean_IBIs = mean(IBIs[1:])
stdev_IBIs = stdev(IBIs[1:])
print('mean IBIs: ', mean_IBIs)
print('std IBIs: ', stdev_IBIs)
print('outliers_minmax: ', len([x for x in IBIs[1:] if x < 0.25 or x > 2]))
print('outliers_std: ', len([x for x in IBIs[1:] if x < mean_IBIs - filter_width * stdev_IBIs or x > mean_IBIs + filter_width * stdev_IBIs]))
print('filter max: ', mean_IBIs + filter_width * stdev_IBIs)
print('filter min: ', mean_IBIs - filter_width * stdev_IBIs)
plt.plot(IBIs)
plt.title("Inter-Beat-Intervals")
plt.ylabel('Milliseconds')
plt.show()

### Calculate Heart Rate (HR)

In [ ]:
HR = calculate_HR(peaks=df['peaks_pred'], win_size=10, frame_rate=20, stride=30)
plt.plot(HR[320:])
plt.title("Heart Rate")
plt.ylabel('BPM')
plt.xlabel('Frames')
plt.show()
mean(HR[320:])

### Calculate Heart Rate Variability (HRV)

In [ ]:
print('RMSSD: ', calculate_HRV_rmssd(IBIs))
print('SDNN: ', stdev_IBIs)

In [ ]:
hrvs = calculate_HRV_rmssd_shortterm(peaks=df['peaks_pred'], win_size=300, stride=80, frame_rate=20)
plt.plot(hrvs)
plt.title("Heart Rate Variability")
plt.xlabel('Frames')
plt.ylabel('Milliseconds')
plt.show()

In [ ]:
mae, HR_gt, HR_pred = compute_MAE_HR(signal_pred=df['pulse_pred'][:1188], signal_gt=pulse_gt_downsampled, win_size=16, frame_rate=20, stride=30)
print('MAE: ', mae)

In [ ]:
print(len(df['pulse_pred']))
print(len(pulse_gt_cleaned_downsampled))

In [ ]:
plt.plot(HR_gt, label='ground truth')
plt.plot(HR_pred, label='prediction')
plt.legend()
plt.title("Heart Rate Comparison")
plt.xlabel('Frames')
plt.ylabel('BPM')
plt.show()

In [ ]:
mae, HRV_gt, HRV_pred = compute_MAE_HRV(signal_pred=df['pulse_pred'][:1188], signal_gt=pulse_gt_downsampled, win_size=300, frame_rate=20, stride=80, metric="")

In [ ]:
print(mae)

In [ ]:
plt.plot(HRV_gt, label='ground truth')
plt.plot(HRV_pred, label='prediction')
plt.legend()
plt.title("Heart Rate Variability Comparison")
plt.xlabel('Frames')
plt.ylabel('Milliseconds')
plt.show()

In [ ]:
# seconds instead of milliseconds???